In [ ]:
import h5py
import pandas as pd
import numpy as np

In [ ]:
f = h5py.File('./data/processed-pythia82-lhc13-all-pt1-50k-r1_h022_e0175_t220_nonu_withPars_truth_0.z', 'r')
treeArray = f['t_allpar_new'][()]

features = ['j1_px', 'j1_py', 'j1_pz', 'j1_e', 'j1_erel', 'j1_pt', 'j1_ptrel', 'j1_eta', 'j1_etarel', 'j1_etarot', 'j1_phi', 'j1_phirel', 'j1_phirot', 'j1_deltaR', 'j1_costheta', 'j1_costhetarel','j_index']
labels = ['j_g','j_q','j_w','j_z','j_t','j_index']

features_df = pd.DataFrame(treeArray,columns=list(set(features)))
labels_df = pd.DataFrame(treeArray,columns=list(set(labels)))
df = pd.concat([features_df.drop(['j_index'],axis=1),labels_df],axis=1)
df = df.reindex(features[:-1]+labels, axis=1)

In [ ]:
def j_transform(MaxParticles, df, features, labels):
    '''
    df should have feature-cols, label-cols and j_index.
    features and labels also should have j_index.
    '''
    df['label'] = df[labels[:-1]].values.tolist()
    df = df.drop(labels[:-1],axis=1)
    
    cols = features[:-1]
    
    jBased_df = pd.DataFrame(columns=cols+['label'])
    jBased_df = jBased_df.astype('object')
    

    for idx, jetId in enumerate(set(df.j_index)):
        nParticles = df[df['j_index']==jetId].shape[0]
        j_df = df[df['j_index']==jetId].sort_values(by=['j1_ptrel'], ascending=False)    # sort descending by ptrel
        
        if nParticles >= MaxParticles:
            j_df = j_df.iloc[:MaxParticles,:]
        else:
            emptyRows = pd.DataFrame(0,index=np.arange(MaxParticles-nParticles),columns = j_df.columns)
            j_df = pd.concat([j_df,emptyRows],axis=0)
            
        for col in cols:
            jBased_df.loc[idx,col] = j_df[col].values
        jBased_df.loc[idx,'label'] = j_df.label.iloc[0]                         

    return jBased_df

In [ ]:
jet_df = j_transform(30,df,features,labels)
jet_df.info()